In [1]:
import pandas as pd
import numpy as np
import re
import ast

# Initial author features

In [2]:
songs = pd.read_csv('songs_dataset.csv', index_col=0)
songs.head()

,Author,Album,Song,Featuring,Lyrics
0,$uicideBoy$,7th or St. Tammany,7th or St. Tammany,[],"\n\nUh\n*59, yah!\nUhhhh, [?]\n*59 motherfucke..."
1,$uicideBoy$,7th or St. Tammany,Dead Batteries,[],\n\n[Produced by Budd Dwyer]\n\n[Intro: Frayse...
2,$uicideBoy$,7th or St. Tammany,I’ll Pay for It (If I Want It),[],\n\n[Verse 1: RUBY DA CHERRY]\nFucking Ruby go...
3,$uicideBoy$,7th or St. Tammany,That’s Very Minimalist of You,[],\n\n[Verse 1: RUBY DA CHERRY]\nRuby casting sh...
4,$uicideBoy$,7th or St. Tammany,Romulus,[],\n\n[Intro: YUNG HANK MOODY]\nGrey*59\nGrey*59...


In [3]:
def format_lyrics(lyrics):
    lyrics = re.sub('[*.,!:?\"\'«»]', '', lyrics)
    lyrics = re.sub('[-–—— ]+', ' ', lyrics)
    lyrics = lyrics.strip()
    lyrics = lyrics.lower()
    return lyrics

In [4]:
agg_funcs = ['max', 'mean', 'median', 'std', 'var']
def get_agg_feats_names(base_name):
    return [f'{base_name}_{agg_func}' for agg_func in agg_funcs]

In [5]:
authors_features = []
for author, author_df in songs.groupby('Author'):
    songs_count = author_df.shape[0]
    albums_count = author_df['Album'].nunique()
    
#     author_df['Lyrics'] = author_df['Lyrics'].apply(format_lyrics)
    
#     all_words = author_df['Lyrics'].apply(lambda x: x.split())
#     all_words_len = all_words.apply(len)
#     all_words_len = all_words_len.agg(agg_funcs)
    
#     all_words_count = all_words.apply(len)
#     unique_words_count = author_df['Lyrics'].apply(lambda x: len(list(set(x.split()))))
#     uniq_words = unique_words_count / all_words_count
#     uniq_words = uniq_words.agg(agg_funcs)
    
    featuring_count = author_df['Featuring'].apply(lambda x: len(ast.literal_eval(x)))
    featuring_count = featuring_count.agg(agg_funcs)
    
#     lyrics_symbols_count = author_df['Lyrics'].apply(len)
#     lyrics_symbols_count = lyrics_symbols_count.agg(agg_funcs)
    
    lyrics_rows_count = author_df['Lyrics'].apply(lambda x: len(x.split('\n')))
    lyrics_rows_count = lyrics_rows_count.agg(agg_funcs)
    
    features = [author, songs_count, albums_count, 
#                 *all_words_len, 
#                 *uniq_words, 
                *featuring_count, 
#                 *lyrics_symbols_count, 
                *lyrics_rows_count]
    authors_features.append(features)

authors = pd.DataFrame(authors_features, columns=['Author', 'songs_count', 'albums_count',
#                                                   *get_agg_feats_names('words_len'),
#                                                   *get_agg_feats_names('uniq_words'),
                                                  *get_agg_feats_names('feats_count'),
#                                                   *get_agg_feats_names('lyrics_symbols_count'),
                                                  *get_agg_feats_names('lyrics_rows_count')])
authors.head()

,Author,songs_count,albums_count,feats_count_max,feats_count_mean,feats_count_median,feats_count_std,feats_count_var,lyrics_rows_count_max,lyrics_rows_count_mean,lyrics_rows_count_median,lyrics_rows_count_std,lyrics_rows_count_var
0,$uicideBoy$,191,30,3.0,0.151832,0.0,0.438879,0.192615,169.0,53.308901,51.0,23.387668,546.983026
1,2 Chainz,207,21,7.0,0.700483,0.0,0.912241,0.832184,182.0,73.246377,75.0,27.582923,760.817645
2,21 Savage,65,6,2.0,0.323077,0.0,0.562019,0.315865,148.0,83.538462,77.0,21.244909,451.346154
3,2Pac,386,29,5.0,0.735751,1.0,0.881331,0.776744,203.0,88.642487,100.0,43.922620,1929.196528
4,3 Doors Down,77,7,1.0,0.025974,0.0,0.160101,0.025632,86.0,54.571429,54.0,10.952563,119.958647


In [6]:
authors.shape

(432, 13)

In [ ]:
authors.to_csv('authors_dataset.csv')

# Author features from parts

In [7]:
parts = pd.read_csv('parts_dataset.csv', index_col=0)
parts.head()

,Author,Album,Song,All Singers,Part Singer,Part Name,Part Lyrics
0,$uicideBoy$,7th or St. Tammany,Dead Batteries,['$uicideBoy$'],$uicideBoy$,Intro,"Hit the, hit the, hit the—\nHit the tone, cock..."
1,$uicideBoy$,7th or St. Tammany,Dead Batteries,['$uicideBoy$'],$uicideBoy$,Verse,Bitches know I got the vendetta\nRollin' up th...
2,$uicideBoy$,7th or St. Tammany,Dead Batteries,['$uicideBoy$'],$uicideBoy$,Hook,"Hit the tone, cock it back, ho, I gotta holla ..."
3,$uicideBoy$,7th or St. Tammany,Dead Batteries,['$uicideBoy$'],$uicideBoy$,Verse,Here I come to paint it grey\nWhip look like I...
4,$uicideBoy$,7th or St. Tammany,Dead Batteries,['$uicideBoy$'],$uicideBoy$,Hook,"Hit the tone, cock it back, ho, I gotta holla ..."


In [14]:
parts['Part Name'].unique()

array(['Intro', 'Verse', 'Hook', 'Outro', 'Chorus', 'Other', 'Bridge',
       'Refrain', nan, 'Skit'], dtype=object)

In [8]:
def format_lyrics(lyrics):
    lyrics = re.sub('[*.’,!:?\"\'«»]', '', lyrics)
    lyrics = re.sub('[-–—— \s]+', ' ', lyrics)
    lyrics = lyrics.strip()
    lyrics = lyrics.lower()
    return lyrics

In [15]:
def get_part_features(part_df, part_name):
    part_name['Part Lyrics'] = part_name['Part Lyrics'].apply(format_lyrics)

    all_words = part_name['Part Lyrics'].apply(lambda x: x.split())
    all_words_len = all_words.apply(len)
    all_words_len = all_words_len.agg(agg_funcs)

    all_words_count = all_words.apply(len)
    unique_words_count = part_name['Part Lyrics'].apply(lambda x: len(list(set(x.split()))))
    uniq_words = unique_words_count / all_words_count
    uniq_words = uniq_words.agg(agg_funcs)

     

authors_features = []

for author, author_df in parts.groupby('Part Singer'):
    parts_freq = author_df['Part Name'].value_counts(normalize=True, dropna=False)
    parts_freq_feats = parts_freq[['Verse', 'Chorus', 'Intro', 'Hook', 
                                   'Outro', 'Bridge', 'Refrain', 'Skit', 'Other', np.nan]].fillna(0).values[:-2]
    verse_features = get_part_features(author_df[author_df['Part Name'] == 'Verse'], 'Verse')
    chorus_features = get_part_features(author_df[author_df['Part Name'] == 'Chorus'], 'Chorus')

    print(author, *parts_freq_feats)
#     authors_features.append()

$teven Cannon 0.7142857142857143 0.14285714285714285 0.14285714285714285 0.0 0.0 0.0 0.0 0.0
$uicideBoy$ 0.5976331360946746 0.03994082840236687 0.11538461538461539 0.13313609467455623 0.08875739644970414 0.01775147928994083 0.0014792899408284023 0.0
070 Shake 0.16666666666666666 0.5 0.3333333333333333 0.0 0.0 0.0 0.0 0.0
1 Life 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
104 0.4444444444444444 0.2222222222222222 0.1111111111111111 0.0 0.0 0.2222222222222222 0.0 0.0
112 0.1111111111111111 0.4444444444444444 0.0 0.3333333333333333 0.0 0.1111111111111111 0.0 0.0
2 Chainz 0.5743519781718963 0.09822646657571624 0.09822646657571624 0.14733969986357434 0.04638472032742155 0.023192360163710776 0.001364256480218281 0.001364256480218281
21 Savage 0.4982698961937716 0.17993079584775087 0.1245674740484429 0.1522491349480969 0.031141868512110725 0.010380622837370242 0.0034602076124567475 0.0
211 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
24hrs 0.25 0.3333333333333333 0.08333333333333333 0.0 0.16666666666666666 0.16666666

Avenged Sevenfold 0.4703703703703704 0.3037037037037037 0.020370370370370372 0.001851851851851852 0.06296296296296296 0.11296296296296296 0.009259259259259259 0.0
Avery Storm 0.2222222222222222 0.1111111111111111 0.0 0.3333333333333333 0.1111111111111111 0.2222222222222222 0.0 0.0
Avonlea 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0
Avril Lavigne 0.3228643216080402 0.44095477386934673 0.02386934673366834 0.0037688442211055275 0.07412060301507538 0.11934673366834171 0.002512562814070352 0.0
Axel Rudi Pell 0.3064516129032258 0.4274193548387097 0.008064516129032258 0.004032258064516129 0.10483870967741936 0.028225806451612902 0.0 0.0
Ayah Marar 0.5 0.0 0.0 0.25 0.0 0.25 0.0 0.0
Ayomari 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Azealia Banks 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Azekel 0.5 0.5 0.0 0.0 0.0 0.0 0.0 0.0
Azizi 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Azizi Gibson 0.43859649122807015 0.13596491228070176 0.08771929824561403 0.21052631578947367 0.07236842105263158 0.04824561403508772 0.0 0.0021929824561403508
B 1.0 0

Bob 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Bob Dylan 0.8971553610503282 0.061269146608315096 0.010393873085339168 0.0005470459518599562 0.009299781181619256 0.012035010940919038 0.007658643326039387 0.0
Bob Marley 0.4714285714285714 0.2857142857142857 0.04285714285714286 0.05 0.07857142857142857 0.014285714285714285 0.0 0.0
Bob Marley and The Wailers 0.5242718446601942 0.28883495145631066 0.050970873786407765 0.0 0.07281553398058252 0.04854368932038835 0.0 0.0
Bob Seger 0.4 0.6 0.0 0.0 0.0 0.0 0.0 0.0
Bobby Brown 0.2857142857142857 0.42857142857142855 0.0 0.0 0.0 0.2857142857142857 0.0 0.0
Bobby Shmurda 0.2 0.0 0.2 0.2 0.2 0.0 0.0 0.0
Bobby V 0.25 0.5 0.125 0.125 0.0 0.0 0.0 0.0
Bobby Womack 0.7142857142857143 0.0 0.0 0.2857142857142857 0.0 0.0 0.0 0.0
Bodega Bamz 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Boldy James 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
BollywoodFM 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Bon Iver 0.0 0.0 0.3333333333333333 0.0 0.6666666666666666 0.0 0.0 0.0
Bon Jovi 0.48546042003231016 0.3675282714

Chloe Angelides 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0
Cho-Flo 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0
Choclatt Jared 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0
Chocolate Tai 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Choo Choo 0.5 0.0 0.0 0.5 0.0 0.0 0.0 0.0
Chris Brown 0.3059948510481795 0.4913571165869805 0.05479955866127253 0.013975726369988967 0.05001838911364472 0.07539536594336153 0.004045605001838912 0.0
Chris Davis 0.5 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Chris Jones (UK) 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Chris King 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Chris Martin 0.0 0.6666666666666666 0.0 0.1111111111111111 0.2222222222222222 0.0 0.0 0.0
Chris Richardson 0.0 0.0 0.5 0.5 0.0 0.0 0.0 0.0
Chris Rivers 0.3333333333333333 0.0 0.0 0.0 0.3333333333333333 0.3333333333333333 0.0 0.0
Chris Rock 0.0 0.5 0.3333333333333333 0.0 0.16666666666666666 0.0 0.0 0.0
Chris Stapleton 0.5 0.5 0.0 0.0 0.0 0.0 0.0 0.0
Chris Travis 0.6666666666666666 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Chris Willis 0.0 0.8 0.0 0.0 0.2 0.0 0.0 0.0
Chrisette Michele 0.0 0.0 0.1

Dee Cosey 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Dee Tha Mad Bitch 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Deep Purple 0.545045045045045 0.3108108108108108 0.024024024024024024 0.003003003003003003 0.07357357357357357 0.03303303303303303 0.0 0.0
Defari 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Deion 0.0 0.5 0.0 0.0 0.5 0.0 0.0 0.0
Delilah Montagu 0.3333333333333333 0.6666666666666666 0.0 0.0 0.0 0.0 0.0 0.0
Dem Jointz 0.0 0.0 0.0 0.6666666666666666 0.3333333333333333 0.0 0.0 0.0
Demarco 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0
Demi Lovato 0.3333333333333333 0.3333333333333333 0.3333333333333333 0.0 0.0 0.0 0.0 0.0
Demrick 0.2857142857142857 0.0 0.0 0.14285714285714285 0.0 0.0 0.0 0.0
Den Bro 0.5 0.0 0.25 0.25 0.0 0.0 0.0 0.0
Denzel Curry 0.4553072625698324 0.2430167597765363 0.07262569832402235 0.10893854748603352 0.08659217877094973 0.0223463687150838 0.0 0.00558659217877095
Depeche Mode 0.4648023143683703 0.36451301832208294 0.01832208293153327 0.011571841851494697 0.0703953712632594 0.06075216972034716 0.00867888138

...

Roots Manuva 0.5 0.0 0.5 0.0 0.0 0.0 0.0 0.0
Rory Fresco 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Rosalee Pfeffer 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Rosario Dawson 0.0 0.0 0.3333333333333333 0.0 0.3333333333333333 0.3333333333333333 0.0 0.0
Roscoe Dash 0.125 0.125 0.0 0.625 0.125 0.0 0.0 0.0
Roscoe Umali 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Rowdy Rebel 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Roxanne Shanté 0.3333333333333333 0.0 0.0 0.3333333333333333 0.0 0.0 0.0 0.0
Royce 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Royce Da 5'9" 0.490216271884655 0.10504634397528322 0.13079299691040164 0.13903192584963955 0.05149330587023687 0.01956745623069001 0.016477857878475798 0.0020597322348094747
Royce da 5'9'’ 0.3333333333333333 0.16666666666666666 0.0 0.3333333333333333 0.0 0.0 0.0 0.0
Roz 0.4 0.0 0.0 0.2 0.0 0.0 0.0 0.0
Rozz Dyliams 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
RuPaul 0.6666666666666666 0.0 0.3333333333333333 0.0 0.0 0.0 0.0 0.0
Ruben Slikk 0.8 0.0 0.0 0.2 0.0 0.0 0.0 0.0
Ruki Vverh 0.423728813559322 0.3728813559322034 0.084

Slick Rick 0.25 0.5 0.0 0.0 0.0 0.25 0.0 0.0
Slim 0.5505050505050505 0.23737373737373738 0.045454545454545456 0.13131313131313133 0.025252525252525252 0.010101010101010102 0.0 0.0
Slim Jxmmi 0.25 0.5 0.25 0.0 0.0 0.0 0.0 0.0
Slim The Mobster 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Slim Thug 0.8181818181818182 0.09090909090909091 0.09090909090909091 0.0 0.0 0.0 0.0 0.0
Slim of 112 0.4 0.2 0.2 0.0 0.2 0.0 0.0 0.0
Slimm Calhoun 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Slipknot 0.3777038269550749 0.3261231281198003 0.04326123128119801 0.0016638935108153079 0.11148086522462562 0.11647254575707154 0.009983361064891847 0.0
Slug Christ 0.5 0.0 0.0 0.5 0.0 0.0 0.0 0.0
Sly Jordan 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0
Sly Pyper 0.5 0.25 0.25 0.0 0.0 0.0 0.0 0.0
Smackola 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Smallz Money 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
SmitBeat 0.5 0.0 0.0 0.0 0.0 0.5 0.0 0.0
Smoke DZA 0.75 0.0 0.25 0.0 0.0 0.0 0.0 0.0
Smokepurpp 0.6 0.2 0.2 0.0 0.0 0.0 0.0 0.0
Smokey Legendary 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0

The Beatles 0.5315712187958884 0.2745961820851689 0.02936857562408223 0.02276064610866373 0.07048458149779736 0.05066079295154185 0.005139500734214391 0.0
The Black Eyed Peas 0.41708542713567837 0.19095477386934673 0.09170854271356783 0.11557788944723618 0.056532663316582916 0.0942211055276382 0.008793969849246231 0.0
The Black Keys 0.5798319327731093 0.3088235294117647 0.012605042016806723 0.0 0.058823529411764705 0.03361344537815126 0.0 0.0
The Breakfast Club (Power 105.1) 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
The Cataracs 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0
The Chainsmokers 0.2206896551724138 0.4724137931034483 0.013793103448275862 0.017241379310344827 0.041379310344827586 0.0896551724137931 0.017241379310344827 0.0
The Chemical Brothers 0.3888888888888889 0.16666666666666666 0.0 0.1111111111111111 0.1111111111111111 0.1111111111111111 0.0 0.0
The Chosen Generation 0.3333333333333333 0.0 0.0 0.0 0.0 0.0 0.0 0.0
The Clark Sisters 0.16666666666666666 0.16666666666666666 0.0 0.0 0.16666666666666

Vado 0.8333333333333334 0.05555555555555555 0.05555555555555555 0.05555555555555555 0.0 0.0 0.0 0.0
Val Young 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0
Valee 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Van Morrison 0.6 0.2 0.0 0.0 0.0 0.2 0.0 0.0
Vanessa Marquez 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Vanilla Ice 0.6205357142857143 0.08482142857142858 0.03571428571428571 0.20535714285714285 0.022321428571428572 0.013392857142857142 0.0 0.0
Vast Aire 0.5 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Vee Tha Rula 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Vein 0.5 0.0 0.0 0.5 0.0 0.0 0.0 0.0
Velous 0.3333333333333333 0.3333333333333333 0.3333333333333333 0.0 0.0 0.0 0.0 0.0
VerBS 0.5714285714285714 0.0 0.0 0.2857142857142857 0.14285714285714285 0.0 0.0 0.0
Verse Simmonds 0.16666666666666666 0.2777777777777778 0.05555555555555555 0.3888888888888889 0.0 0.1111111111111111 0.0 0.0
Vic Mensa 0.3333333333333333 0.0 0.3333333333333333 0.0 0.0 0.3333333333333333 0.0 0.0
Vice Versa 0.3333333333333333 0.3333333333333333 0.3333333333333333 0.0 0.0 0.0 0.0

Влади (Vlady) 0.0 0.3333333333333333 0.3333333333333333 0.0 0.3333333333333333 0.0 0.0 0.0
Гуф (Guf) 0.0 0.0 0.3333333333333333 0.0 0.3333333333333333 0.0 0.0 0.3333333333333333
Касим (Kasym) 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0
Каспийский Груз (KG) 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0
Кастро (Kastro RUS) 0.6666666666666666 0.0 0.0 0.3333333333333333 0.0 0.0 0.0 0.0
Крестная Семья (Krestnaya Semya) 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0
Лёха Никонов (Lyokha Nikonov) 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Надя Дорофеева  (Nadya Dorofeeva) 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0
Нервы (Nervy) 0.5 0.5 0.0 0.0 0.0 0.0 0.0 0.0
Ноггано (Noggano) 0.5 0.25 0.25 0.0 0.0 0.0 0.0 0.0
ОУ74 (OU74) 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Олег Груз (Oleg Gruz) 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0
Принцип 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Принцип (Princip) 0.4 0.0 0.0 0.2 0.0 0.0 0.0 0.0
Птаха (Ptaha) 0.0 0.75 0.0 0.0 0.25 0.0 0.0 0.0
Птица (Ptitca) 0.75 0.25 0.0 0.0 0.0 0.0 0.0 0.0
РУБЛЬ (RUBL) 0.2 0.6 0.0 0.0 0.0 0.0 0.0 0.2
Ринат Султанов (Rina

In [ ]:
parts[parts['Author'] == 'Eminem']